In [ ]:
%pip install grad-cam

In [13]:
import random
from tqdm import tqdm

In [2]:
import kagglehub
import os

# --- Celeb-DF v2 ---
celeb_df_path = kagglehub.dataset_download("reubensuju/celeb-df-v2")
print("Celeb-DF v2 path:", celeb_df_path)
print("Celeb-DF v2 folders:", os.listdir(celeb_df_path))

Celeb-DF v2 path: C:\Users\mujta\.cache\kagglehub\datasets\reubensuju\celeb-df-v2\versions\1
Celeb-DF v2 folders: ['Celeb-real', 'Celeb-synthesis', 'List_of_testing_videos.txt', 'YouTube-real']


In [3]:
# --- FaceForensics++ (Kaggle mirror) ---
ffpp_path = kagglehub.dataset_download("xdxd003/ff-c23")
print("FF++ path:", ffpp_path)
print("FF++ folders:", os.listdir(ffpp_path))

FF++ path: C:\Users\mujta\.cache\kagglehub\datasets\xdxd003\ff-c23\versions\1
FF++ folders: ['FaceForensics++_C23']


In [8]:
# Download VGGFace2 dataset
path_vgg = kagglehub.dataset_download("hearfool/vggface2")
print("VGGFace2 dataset path:", path_vgg)

100%|█████████████████████████████████████████████████████████████████████████████| 2.32G/2.32G [04:27<00:00, 9.33MB/s]

Extracting files...


VGGFace2 dataset path: C:\Users\mujta\.cache\kagglehub\datasets\hearfool\vggface2\versions\1


In [9]:
# Show the folder structure of Celeb DF
for root, dirs, files in os.walk(celeb_df_path):
    level = root.replace(ffpp_path, "").count(os.sep)
    indent = " " * 4 * (level)
    print(f"{indent}{os.path.basename(root)}/")
    subindent = " " * 4 * (level + 1)
    for f in files[:10]:  # show only first 10 files per directory
        print(f"{subindent}{f}")
    if len(files) > 10:
        print(f"{subindent}... ({len(files)} files total)")

# Show the folder structure of FF++
for root, dirs, files in os.walk(ffpp_path):
    level = root.replace(ffpp_path, "").count(os.sep)
    indent = " " * 4 * (level)
    print(f"{indent}{os.path.basename(root)}/")
    subindent = " " * 4 * (level + 1)
    for f in files[:10]:  # show only first 10 files per directory
        print(f"{subindent}{f}")
    if len(files) > 10:
        print(f"{subindent}... ({len(files)} files total)")

# Show the folder structure of VGGFace2
for root, dirs, files in os.walk(path_vgg):
    level = root.replace(ffpp_path, "").count(os.sep)
    indent = " " * 4 * (level)
    print(f"{indent}{os.path.basename(root)}/")
    subindent = " " * 4 * (level + 1)
    for f in files[:10]:  # show only first 10 files per directory
        print(f"{subindent}{f}")
    if len(files) > 10:
        print(f"{subindent}... ({len(files)} files total)")

                                    1/
                                        List_of_testing_videos.txt
                                        Celeb-real/
                                            id0_0000.mp4
                                            id0_0001.mp4
                                            id0_0002.mp4
                                            id0_0003.mp4
                                            id0_0004.mp4
                                            id0_0005.mp4
                                            id0_0006.mp4
                                            id0_0007.mp4
                                            id0_0008.mp4
                                            id0_0009.mp4
                                            ... (590 files total)
                                        Celeb-synthesis/
                                            id0_id16_0000.mp4
                                            id0_id16_0001.mp4
                         

## PRE PROCESSING

In [6]:
import os
import cv2
import torch
from mtcnn import MTCNN
from torchvision import transforms
from PIL import Image

# Paths
CELEB_DF_PATH = r"C:\Users\mujta\.cache\kagglehub\datasets\reubensuju\celeb-df-v2\versions\1"
FFPP_PATH     = r"C:\Users\mujta\.cache\kagglehub\datasets\xdxd003\ff-c23\versions\1\FaceForensics++_C23"

# Output directory
OUTPUT_PATH = "processed_faces"
os.makedirs(OUTPUT_PATH, exist_ok=True)

# MTCNN face detector
detector = MTCNN()

# Preprocessing
preprocess_tf = transforms.Compose([
    transforms.Resize((299, 299)),  # For Inception
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def extract_faces_from_video(video_path, label, save_dir, frame_skip=30):
    cap = cv2.VideoCapture(video_path)
    frame_idx, saved = 0, 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_idx % frame_skip == 0:
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            detections = detector.detect_faces(rgb)
            if len(detections) > 0:
                # largest face
                face = max(detections, key=lambda x: x['box'][2]*x['box'][3])
                x, y, w, h = face['box']
                margin = 0.2
                x1 = max(0, int(x - w*margin))
                y1 = max(0, int(y - h*margin))
                x2 = min(frame.shape[1], int(x + w*(1+margin)))
                y2 = min(frame.shape[0], int(y + h*(1+margin)))
                face_crop = rgb[y1:y2, x1:x2]
                if face_crop.size == 0:
                    continue
                pil_face = Image.fromarray(face_crop)
                face_tensor = preprocess_tf(pil_face)
                save_path = os.path.join(save_dir, f"{label}_{os.path.basename(video_path)}_{frame_idx}.pt")
                torch.save(face_tensor, save_path)
                saved += 1
        frame_idx += 1
    cap.release()
    return saved


### ---- Celeb-DF ----
real_dir = os.path.join(CELEB_DF_PATH, "Celeb-real")
fake_dir = os.path.join(CELEB_DF_PATH, "Celeb-synthesis")
yt_dir   = os.path.join(CELEB_DF_PATH, "YouTube-real")

save_real = os.path.join(OUTPUT_PATH, "real")
save_fake = os.path.join(OUTPUT_PATH, "fake")
os.makedirs(save_real, exist_ok=True)
os.makedirs(save_fake, exist_ok=True)

print("Extracting Celeb-DF faces...")
for video in os.listdir(real_dir):
    extract_faces_from_video(os.path.join(real_dir, video), "real", save_real)
for video in os.listdir(yt_dir):
    extract_faces_from_video(os.path.join(yt_dir, video), "real", save_real)
for video in os.listdir(fake_dir):
    extract_faces_from_video(os.path.join(fake_dir, video), "fake", save_fake)

### ---- FF++ ----
ff_dirs = {
    "real": os.path.join(FFPP_PATH, "original"),
    "fake": [
        os.path.join(FFPP_PATH, "Deepfakes"),
        os.path.join(FFPP_PATH, "Face2Face"),
        os.path.join(FFPP_PATH, "FaceShifter"),
        os.path.join(FFPP_PATH, "FaceSwap"),
        os.path.join(FFPP_PATH, "NeuralTextures"),
    ]
}

print("Extracting FF++ faces...")
# Real
for video in os.listdir(ff_dirs["real"]):
    extract_faces_from_video(os.path.join(ff_dirs["real"], video), "real", save_real)

# Fakes (multiple methods)
for fdir in ff_dirs["fake"]:
    for video in os.listdir(fdir):
        extract_faces_from_video(os.path.join(fdir, video), "fake", save_fake)

print("✅ Extraction finished!")


Extracting Celeb-DF faces...
Extracting FF++ faces...
✅ Extraction finished!


In [19]:
import os
import torch
from tqdm import tqdm
from PIL import Image
import torchvision.transforms as T

# Paths
vggface2_train_dir = r"C:\Users\mujta\.cache\kagglehub\datasets\hearfool\vggface2\versions\1\train"
real_dir = r"processed_faces\real"

# Current stats
existing_real = len([f for f in os.listdir(real_dir) if f.endswith('.pt')])
target_total = 94367  # equal to your fake count
need_to_add = max(0, target_total - existing_real)

print(f"Existing real images: {existing_real}")
print(f"Need to add {need_to_add} from VGGFace2 to balance with fakes.")

if need_to_add <= 0:
    print("✅ Already balanced.")
else:
    # Transform (same as before)
    transform = T.Compose([
        T.Resize((299, 299)),  # Match model input (Inception-based)
        T.ToTensor()
    ])

    added = 0
    failed = 0
    failed_log = open("failed_vggface2_images.txt", "w")

    # Iterate over all identities
    for person in os.listdir(vggface2_train_dir):
        person_path = os.path.join(vggface2_train_dir, person)
        if not os.path.isdir(person_path):
            continue

        for file in tqdm(os.listdir(person_path), desc=f"Converting {person}", leave=False):
            if not file.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue

            src = os.path.join(person_path, file)
            dest = os.path.join(real_dir, f"{person}_{file[:-4]}_vgg.pt")

            if os.path.exists(dest):
                continue

            try:
                img = Image.open(src).convert("RGB")
                tensor = transform(img)
                torch.save(tensor, dest)
                added += 1
            except Exception as e:
                failed += 1
                failed_log.write(f"{src} | {e}\n")

            if added >= need_to_add:
                break

        if added >= need_to_add:
            break

    failed_log.close()
    print(f"✅ Added {added} new real faces.")
    print(f"⚠️ Failed: {failed} images (see failed_vggface2_images.txt).")

# Final check
final_total = len([f for f in os.listdir(real_dir) if f.endswith('.pt')])
print(f"📊 Final total real images: {final_total}")

Existing real images: 33097
Need to add 61270 from VGGFace2 to balance with fakes.


✅ Added 61270 new real faces.
⚠️ Failed: 0 images (see failed_vggface2_images.txt).
📊 Final total real images: 94367


# BUILD DATASET AND AUGMENTATIONS

In [1]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import random
import io
from PIL import Image

# =======================================
# 1️⃣  Custom Dataset for .pt tensors
# =======================================
class DeepfakeDataset(Dataset):
    def __init__(self, real_dir, fake_dir, transform=None):
        self.real_paths = [os.path.join(real_dir, f) for f in os.listdir(real_dir) if f.endswith('.pt')]
        self.fake_paths = [os.path.join(fake_dir, f) for f in os.listdir(fake_dir) if f.endswith('.pt')]
        self.samples = [(p, 0) for p in self.real_paths] + [(p, 1) for p in self.fake_paths]
        random.shuffle(self.samples)
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img_tensor = torch.load(path)

        # convert to PIL for augmentations that require it
        img = T.ToPILImage()(img_tensor)

        if self.transform:
            img = self.transform(img)

        return img, torch.tensor(label, dtype=torch.long)

# =======================================
# 2️⃣  Augmentation pipeline
# =======================================
class RandomJPEGCompression:
    """Simulates random JPEG compression artifacts."""
    def __init__(self, quality=(20, 100)):
        self.quality = quality

    def __call__(self, img):
        buffer = io.BytesIO()
        q = random.randint(*self.quality)
        img.save(buffer, format="JPEG", quality=q)
        buffer.seek(0)
        return Image.open(buffer)

augmentations = T.Compose([
    T.RandomApply([RandomJPEGCompression((20, 100))], p=0.5),
    T.RandomApply([T.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0))], p=0.3),
    T.RandomResizedCrop(299, scale=(0.8, 1.0), ratio=(0.9, 1.1)),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(degrees=5),
    T.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.02),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
])

# =======================================
# 3️⃣  Train / Val split
# =======================================
real_dir = "sampled_faces/real"
fake_dir = "sampled_faces/fake"

full_dataset = DeepfakeDataset(real_dir, fake_dir, transform=augmentations)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# =======================================
# 4️⃣  Dataloaders
# =======================================
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

print(f"✅ Dataset ready: {len(train_dataset)} train, {len(val_dataset)} val")

✅ Dataset ready: 32000 train, 8000 val


## MODEL TRAINING

In [1]:
import torch, os, random, shutil
from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image

# Paths
real_dir = "sampled_faces/real"
fake_dir = "sampled_faces/fake"
output_root = "sampled_faces"

# Create subfolders
for split in ["train", "val", "test"]:
    for cls in ["real", "fake"]:
        os.makedirs(os.path.join(output_root, split, cls), exist_ok=True)

# Get all .pt paths
real_files = [os.path.join(real_dir, f) for f in os.listdir(real_dir) if f.endswith(".pt")]
fake_files = [os.path.join(fake_dir, f) for f in os.listdir(fake_dir) if f.endswith(".pt")]

# Shuffle
random.shuffle(real_files)
random.shuffle(fake_files)

def split_data(files):
    n = len(files)
    train = files[:int(0.8*n)]
    val = files[int(0.8*n):int(0.9*n)]
    test = files[int(0.9*n):]
    return train, val, test

real_splits = split_data(real_files)
fake_splits = split_data(fake_files)

# Copy or link .pt files
def copy_split(files, split, cls):
    for src in tqdm(files, desc=f"{cls}-{split}"):
        dst = os.path.join(output_root, split, cls, os.path.basename(src))
        if not os.path.exists(dst):
            shutil.copy(src, dst)

for split_idx, split_name in enumerate(["train", "val", "test"]):
    copy_split(real_splits[split_idx], split_name, "real")
    copy_split(fake_splits[split_idx], split_name, "fake")

print("✅ Dataset split done.")


fake-test: 100%|███████████████████████████████████████████████████████████████████| 2000/2000 [00:50<00:00, 39.73it/s]

✅ Dataset split done.


In [2]:
from torchvision import transforms

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(299, scale=(0.9, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3)], p=0.5),
    transforms.RandomApply([transforms.GaussianBlur(3)], p=0.3),
    transforms.RandomRotation(5),
    transforms.ToTensor(),
    transforms.RandomApply([transforms.Lambda(lambda x: x + 0.02 * torch.randn_like(x))], p=0.3),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

val_test_transform = transforms.Compose([
    transforms.Resize((299,299)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])


In [ ]:
import torch
import torch.nn as nn
from torchvision import models
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

class DeepfakeDataset(Dataset):
    def __init__(self, root_dir, transform):
        self.paths = []
        self.labels = []
        for label, cls in enumerate(["real", "fake"]):
            folder = os.path.join(root_dir, cls)
            for f in os.listdir(folder):
                if f.endswith(".pt"):
                    self.paths.append(os.path.join(folder, f))
                    self.labels.append(label)
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        tensor = torch.load(self.paths[idx])  # shape: C,H,W
        img = transforms.ToPILImage()(tensor)
        if self.transform:
            img = self.transform(img)
        label = self.labels[idx]
        return img, label

# Build model
def build_inceptionv3(num_classes=2):
    model = models.inception_v3(weights="IMAGENET1K_V1")
    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 512),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(512, num_classes)
    )
    return model

# Load data
train_ds = DeepfakeDataset("sampled_faces/train", train_transform)
val_ds   = DeepfakeDataset("sampled_faces/val", val_test_transform)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=4)

# Initialize
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = build_inceptionv3().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, factor=0.5)

# Training
best_acc = 0
for epoch in range(10):
    model.train()
    train_loss, train_correct = 0, 0
    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/10 [Train]"):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        out = model(imgs)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * imgs.size(0)
        train_correct += (out.argmax(1) == labels).sum().item()
    train_acc = train_correct / len(train_loader.dataset)

    # Validation
    model.eval()
    val_correct, val_loss = 0, 0
    with torch.no_grad():
        for imgs, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/10 [Val]"):
            imgs, labels = imgs.to(device), labels.to(device)
            out = model(imgs)
            loss = criterion(out, labels)
            val_loss += loss.item() * imgs.size(0)
            val_correct += (out.argmax(1) == labels).sum().item()
    val_acc = val_correct / len(val_loader.dataset)

    scheduler.step(val_loss)

    print(f"Epoch {epoch+1}: TrainAcc={train_acc:.4f}, ValAcc={val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_inceptionv3.pth")

print("✅ Training complete! Best Val Acc:", best_acc)

Epoch 1/10 [Train]:   0%|                                                                     | 0/1000 [00:00<?, ?it/s]

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import Dataset, DataLoader, random_split
import os
from tqdm import tqdm

# ==============================
# 1️⃣ Dataset Loader (.pt)
# ==============================
class DeepfakeDataset(Dataset):
    def __init__(self, real_dir, fake_dir):
        self.paths = []
        self.labels = []
        for folder, label in [(real_dir, 0), (fake_dir, 1)]:
            for f in os.listdir(folder):
                if f.endswith(".pt"):
                    self.paths.append(os.path.join(folder, f))
                    self.labels.append(label)

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        x = torch.load(self.paths[idx])
        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return x, y

# ==============================
# 2️⃣ Load Full Dataset
# ==============================
real_dir = "sampled_faces/real"
fake_dir = "sampled_faces/fake"

full_dataset = DeepfakeDataset(real_dir, fake_dir)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

print(f"✅ Dataset ready: {len(train_dataset)} train, {len(val_dataset)} val")

# ==============================
# 3️⃣ Model Setup (InceptionV3)
# ==============================
def build_inceptionv3(num_classes=2):
    model = models.inception_v3(weights="IMAGENET1K_V1")
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

# ==============================
# 4️⃣ Training Function
# ==============================
def evaluate(model, loader, criterion, device):
    model.eval()
    loss_sum, correct, total = 0, 0, 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss_sum += loss.item() * imgs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total, loss_sum / total

def train_inception(model, train_loader, val_loader, device, epochs=20):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    best_val_acc, patience, wait = 0, 3, 0

    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0

        for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]"):
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * imgs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total
        val_acc, val_loss = evaluate(model, val_loader, criterion, device)
        scheduler.step()

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {total_loss/total:.4f} | Train Acc: {train_acc:.4f}")
        print(f"                 Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_inceptionv3.pth")
            print("✅ Saved best model.")
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                print("⏹️ Early stopping.")
                break

    print(f"🏁 Training complete. Best Val Acc: {best_val_acc:.4f}")

✅ Dataset ready: 32000 train, 8000 val


In [ ]:
# ==============================
# 5️⃣ Run Training
# ==============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = build_inceptionv3().to(device)
train_inception(model, train_loader, val_loader, device, epochs=20)

Epoch 1/20 [Train]:   0%|                                                                     | 0/1000 [00:00<?, ?it/s]

## EVALUATION AND TESTING

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---- Load model ----
def build_inceptionv3(num_classes=2):
    model = models.inception_v3(weights=None)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

model = build_inceptionv3().to(device)
model.load_state_dict(torch.load("best_inceptionv3.pth", map_location=device))
model.eval()

# ---- Evaluate ----
all_labels, all_preds, all_probs = [], [], []

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        probs = torch.softmax(outputs, dim=1)[:, 1]
        preds = torch.argmax(outputs, dim=1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

print("✅ Evaluation done!")

In [ ]:
print("\n--- Classification Report ---")
print(classification_report(all_labels, all_preds, target_names=["Real", "Fake"]))

cm = confusion_matrix(all_labels, all_preds)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Pred Real", "Pred Fake"],
            yticklabels=["True Real", "True Fake"])
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

In [ ]:
fpr, tpr, _ = roc_curve(all_labels, all_probs)
auc_score = roc_auc_score(all_labels, all_probs)

plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {auc_score:.3f})")
plt.plot([0, 1], [0, 1], "--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.title("ROC Curve – Inception V3")
plt.show()

print(f"ROC AUC Score: {auc_score:.4f}")

In [ ]:
from PIL import Image
from torchvision import transforms

# Match training preprocessing
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

def predict_image(img_path):
    img = Image.open(img_path).convert("RGB")
    tensor = transform(img).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        output = model(tensor)
        probs = torch.softmax(output, dim=1)[0]
        pred = torch.argmax(probs).item()

    label = "REAL ✅" if pred == 0 else "FAKE 🚫"
    print(f"{img_path} → {label} (Fake prob = {probs[1]:.4f})")

# Run prediction
predict_image("test_images/real_sample.jpg")
predict_image("test_images/spoof_sample.jpg")

# HEAT MAP

In [ ]:
import torch
import numpy as np
import cv2
from torchvision import models, transforms
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from PIL import Image
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---- Load your trained model ----
def build_inceptionv3(num_classes=2):
    model = models.inception_v3(weights=None)
    model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
    return model

model = build_inceptionv3().to(device)
model.load_state_dict(torch.load("best_inceptionv3.pth", map_location=device))
model.eval()

# ---- Choose target layer (final conv block) ----
target_layers = [model.Mixed_7c]

# ---- Transform (same as during training) ----
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# ---- Grad-CAM Function ----
def generate_heatmap(image_path, label_names=["Real", "Fake"]):
    img = Image.open(image_path).convert("RGB")
    rgb_img = np.array(img) / 255.0
    input_tensor = transform(img).unsqueeze(0).to(device)

    cam = GradCAM(model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available())
    targets = [ClassifierOutputTarget(1)]  # Target class: Fake

    grayscale_cam = cam(input_tensor=input_tensor, targets=targets)[0, :]
    visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.imshow(rgb_img)
    plt.title("Original Image")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(visualization)
    plt.title("Grad-CAM Heatmap (Fake regions)")
    plt.axis("off")

    plt.tight_layout()
    plt.show()

# ---- Example ----
generate_heatmap("test_images/spoof_sample.jpg")